In [2]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd
from pandas import DataFrame

df = pd.read_excel (r'D:\KULIAH\Tugas Kuliah\Semester 8\Skripsi\dataset\dataskripsi.xlsx')
df.head()

,Contrast,Energy,Homogenity,Correlation,Mean,St.Deviasi,Skewness,Kurtosis,Area,Perimeter,Solidity,Compactness,Convexity,Target
0,3387035,426756518,20802.757611,2329.671698,67.632548,82.734350,0.860106,2.226210,1374.5,199.639609,0.746605,0.433153,0.835652,0
1,2981494,422635015,20678.925312,2862.578987,70.780192,86.357111,0.729668,1.980532,1404.0,208.367531,0.740116,0.406160,0.806316,0
2,1597889,472281968,21816.576695,721.794651,60.142390,83.427250,1.032554,2.588830,338.0,76.041630,0.946779,0.734182,0.941921,0
3,2887726,452713681,21345.551032,1641.718560,48.443695,82.672852,1.393999,3.089705,402.0,91.941125,0.881579,0.597305,0.940169,0
4,2559642,450376185,21310.081252,660.438792,46.797689,75.350361,1.502072,3.745117,451.0,99.213203,0.941545,0.575476,0.869681,0


In [24]:
X=df.drop(['Target'], axis=1)
# X=df.drop(['Target', 'Area', 'Perimeter','Solidity','Compactness','Convexity'], axis=1)

X.head()
y = df['Target']
y.head()

0    0
1    0
2    0
3    0
4    0
Name: Target, dtype: int64

In [25]:
X_min = X.min()
X_max = X.max()
X_range = (X_max - X_min)
X_scaled = (X - X_min)/(X_range)
X_scaled.head()

,Contrast,Energy,Homogenity,Correlation,Mean,St.Deviasi,Skewness,Kurtosis,Area,Perimeter,Solidity,Compactness,Convexity
0,0.479504,0.682463,0.710422,0.224697,0.751584,0.551135,0.232064,0.103258,0.085818,0.125955,0.746605,0.509156,0.835652
1,0.421879,0.670011,0.694557,0.276134,0.802350,0.657588,0.161533,0.053169,0.087660,0.131461,0.740116,0.477427,0.806316
2,0.225275,0.820009,0.840312,0.069501,0.630779,0.571496,0.325310,0.177190,0.021103,0.047976,0.946779,0.863006,0.941921
3,0.408555,0.760888,0.779964,0.158294,0.442098,0.549328,0.520753,0.279310,0.025099,0.058007,0.881579,0.702112,0.940169
4,0.361935,0.753825,0.775420,0.063579,0.415551,0.334160,0.579190,0.412938,0.028158,0.062595,0.941545,0.676453,0.869681


In [47]:
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC 
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import classification_report
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score



svm_model_linear = SVC(decision_function_shape='ovo', kernel = 'linear', C=10)
skf = StratifiedKFold(n_splits=10, random_state=10 , shuffle=True)
# cv = ShuffleSplit(n_splits=10, random_state=0)

scores2 = cross_val_score(svm_model_linear, X_scaled, y, cv=skf)


print(cv)

scores=[]
i=1
c = np.zeros((3,3)) 

for train_index, test_index in skf.split(X_scaled, y):
#     print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X_scaled.iloc[train_index], X_scaled.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    svm_model_linear.fit(X_train, y_train)
    svm_predict = svm_model_linear.predict(X_test)
    cm = confusion_matrix(y_test, svm_predict)
    cm = np.array(confusion_matrix(y_test, svm_predict, labels=[0,1,2]))
    confusion = pd.DataFrame(cm, index=['early blight', 'late blight', 'healthy'],
                        columns=['predik early', 'prediksi late', 'predik health'])
    
    scores.append(svm_model_linear.score(X_test, y_test))
    print("=============Fold ke-",i,"%s============")
    print(classification_report(y_test, svm_predict), "\n")
    print(confusion, "\n\n")
    c=cm+c
    print(c, "\n\n")

    if (i==7):
        print('akurasi : ', accuracy_score(y_test, svm_predict))
        print('f1=score : ', f1_score(y_test, svm_predict, average="macro"))
        print('presisi :', precision_score(y_test, svm_predict, average="macro"))
        print('recall :', recall_score(y_test, svm_predict, average="macro"))  
    i+=1
    
print("scores 1 : ", scores,"\n")

print("rata-rata :", np.mean(scores))


ShuffleSplit(n_splits=10, random_state=0, test_size=None, train_size=None)
=============Fold ke- 1 %s============
              precision    recall  f1-score   support

           0       1.00      0.93      0.97        15
           1       0.94      1.00      0.97        15
           2       1.00      1.00      1.00        15

    accuracy                           0.98        45
   macro avg       0.98      0.98      0.98        45
weighted avg       0.98      0.98      0.98        45
 

              predik early  prediksi late  predik health
early blight            14              1              0
late blight              0             15              0
healthy                  0              0             15 


[[14.  1.  0.]
 [ 0. 15.  0.]
 [ 0.  0. 15.]] 


=============Fold ke- 2 %s============
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        15
           1       1.00      1.00      1.00        15
           2       1.

In [37]:
import joblib
joblib_file = "SVM_model_linear.pkl" 
joblib.dump(svm_model_linear, joblib_file)
joblib_model = joblib.load(joblib_file)
joblib_model

SVC(C=10, decision_function_shape='ovo', kernel='linear')